In [36]:
import pandas as pd

In [37]:
data_medecin = pd.read_excel("/content/export_medecins_et_correspondants_specialites - Copy.xlsx")

In [38]:
import re
import numpy as np
def search_adresse(text):
  text = str(text)
  text = text.replace(" " ,"")
  pattern_email = r'([\w\.-]+@[A-Za-z\-]+\.(com|ma|fr|net|org|edu|gov|io|co))'

  match = re.search(pattern_email , text)
  if match :
    return True
  return False

def extract_adresse(text):
  text = str(text)
  if search_adresse(text):
    pattern_email = r'([\w\.-]+@[A-Za-z\-]+\.(com|ma|fr|net|org|edu|gov|io|co))'
    match = re.search(pattern_email , text)
    return match.group(1)

  return np.nan


search_adresse('omarb773_oumaize@my-company.com8484')
extract_adresse('omar.b773_oumaize@my-company.com8484')


'omar.b773_oumaize@my-company.com'

In [39]:
print("Null counts in adresse")
print(data_medecin['adresse'].isna().value_counts())
print()
print("Null counts in adresse after extracting")
print(data_medecin['adresse'].apply(extract_adresse).isna().value_counts() )


Null counts in adresse
adresse
True     1236
False     159
Name: count, dtype: int64

Null counts in adresse after extracting
adresse
True    1395
Name: count, dtype: int64


looking for valid email

In [40]:
found_adresse_columns = []
search_adresses_colums = data_medecin.columns.tolist()


data_medecin['id'] = data_medecin['id'].astype(str)



In [41]:
data_medecin['adresse'][data_medecin['adresse'].apply(search_adresse)]

,adresse


In [42]:
columns= data_medecin.drop(columns=['adresse']).columns.tolist()
list_search = []

for col in columns :
  result = data_medecin[col][data_medecin[col].apply(search_adresse)]
  shape = result.shape[0]

  if shape > 1 :
    column = f"adresse_from{col}"
    list_search.append(column)
    data_medecin[column] = result

no adresse found

In [43]:
list_search

['adresse_fromemail']

telephone

In [44]:
def search_fax(text):
  if isinstance(text , str):
    text = text.replace(' ' , '')
    patterns_phone =r'\D'
    filtered = re.sub(patterns_phone,'' , text)
    extract_fax = re.search(r'(\d{9})' , filtered)
    if extract_fax :
      return True
  return False


def extract_fax(text):
  if isinstance(text , str):
    text = text.replace(' ' , '')
    patterns_phone =r'\D'
    filtered = re.sub(patterns_phone,'' , text)
    pattern_fax1 = re.search(r'(?:212)*[+*.:/,-]*(0\d{9})$' , filtered)
    pattern_fax2 = re.search(r'(?:212)*[+*.:/,-]*(\d{9})$' , filtered)

    if pattern_fax1 :
      return pattern_fax1.group(1)

    elif pattern_fax2 :
      return '0' + pattern_fax2.group(1)

  return np.nan
print(extract_fax('CIN : N 159477 / 670249049'))
print(extract_fax('CIN : N 159477 / 212+0670249049'))


0670249049
0670249049


In [45]:
print("phone in adresse")
print(data_medecin['adresse'].apply(search_fax).value_counts())
data_medecin['adresse'][data_medecin['adresse'].apply(search_fax)]

phone in adresse
adresse
False    1395
Name: count, dtype: int64


,adresse


In [46]:
columns= data_medecin.columns.tolist()
list_search = []

for col in columns :
  result = data_medecin[col][data_medecin[col].apply(search_fax)]
  shape = result.shape[0]

  if shape > 1 :
    column = f"telephone_from_{col}"
    list_search.append(column)
    data_medecin[column] = result

In [47]:
data_medecin


for col in list_search :
  mask = data_medecin['telephone1'].isna() & data_medecin[col].notna()
  data_medecin.loc[mask , 'telephone1'] = data_medecin.loc[mask , col]


In [48]:
mask = data_medecin['telephone1'].dropna().apply(extract_fax).index

data_medecin.loc[mask , 'telephone1'] = data_medecin['telephone1'].loc[mask]

In [49]:
data_medecin['telephone1'].dropna()

,telephone1
0,fixe
1,0524473144
4,0659707882
7,0524396566
9,0527957744
...,...
1349,0524792937
1354,0621824578
1371,0659538518
1375,0649258888


groupage data

Table that stores information related to the blood types of patients.

In [50]:
data_groupage = pd.read_excel('/content/export_groupages.xlsx')

In [51]:
data_groupage.columns

Index(['id', 'ref', 'datevisite', 'nom', 'adresse', 'datenaiss',
       'Aff_datenaiss', 'determine1', 'determine2', 'Groupe', 'Rhesus',
       'Phenotype', 'Vignette', 'RefAutomate', 'groupe2', 'Rhesus2',
       'Phenotype2', 'sexe', 'Aff_Groupe', 'Vignette2', 'Rq1', 'Rq2', 'Rq3',
       'Rq4'],
      dtype='object')

Data we need to integrate

In [52]:
data_groupage[['id' , 'Groupe' , 'determine1' , 'determine2' ,'id' , 'Rhesus' , 'Vignette' ]]

,id,Groupe,determine1,determine2,id,Rhesus,Vignette
0,1,A,31/10/2014,NaN,1,POSITIF,A RH POSITIF
1,2,O,01/11/2014,NaN,2,POSITIF,O RH POSITIF
2,3,A,03/11/2014,NaN,3,POSITIF,A RH POSITIF
3,4,O,03/11/2014,NaN,4,POSITIF,O RH POSITIF
4,5,O,03/11/2014,NaN,5,POSITIF,O RH POSITIF
...,...,...,...,...,...,...,...
13309,13316,A,20/11/2024,NaN,13316,POSITIF,A+
13310,13317,O,20/11/2024,NaN,13317,POSITIF,O+
13311,13318,O,20/11/2024,NaN,13318,POSITIF,O+
13312,13319,A,13/07/2019,21/11/2024,13319,POSITIF,A+


Transforming Vignette

In [53]:
import re

def search_vignette(text):
    text = str(text).upper()
    # Don't remove spaces here, keep them for proper matching
    pattern = r'\bP\s*O\s*S\s*I\s*T\s*I\s*F\b|\bN\s*E\s*G\s*A\s*T\s*I\s*F\b|\bR\s*H\b'

    if re.search(pattern, text, flags=re.IGNORECASE):
        return True
    else:
        return False

def transform_vignette(text):
    if search_vignette(text):
        text = str(text).upper()
        # Don't remove spaces before substitution
        def repl(m):
            if m.group(1):  # POSITIF matched
                return '+'
            elif m.group(2):  # NEGATIF matched
                return '-'
            elif m.group(3):  # RH matched
                return ''
            else:
                return m.group(0)

        pattern = r'(\bP\s*O\s*S\s*I\s*T\s*I\s*F\b)|(\bN\s*E\s*G\s*A\s*T\s*I\s*F\b)|(\bR\s*H\b)'
        text = re.sub(pattern, repl, text, flags=re.IGNORECASE)

        # Now remove spaces after replacement
        text = text.replace(' ', '')
        return text

    else:
        return text  # or return original text if no match

print(transform_vignette('O RH POSITIF'))  # Expected output: O+
print(transform_vignette('A RH NEGATIF'))  # Expected output: A-


O+
A-


In [54]:
transformed_Vignette = data_groupage['Vignette'][data_groupage['Vignette'].apply(search_vignette) ].apply(transform_vignette)

data_groupage['Vignette'].loc[transformed_Vignette.index] = transformed_Vignette

<ipython-input-54-1874599877>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_groupage['Vignette'].loc[transformed_Vignette.index] = transformed_Vignette
<ipython-input-54-1874599877>:3: SettingWithCopyWarning: 
A value is trying to be

In [55]:
data_groupage['Vignette'].value_counts()

,count
Vignette,
O+,6094
A+,3932
B+,1738
O-,544
AB+,430
A-,363
B-,164
AB-,49


In [56]:
data_groupage[['determine1' , 'determine2' ]]

,determine1,determine2
0,31/10/2014,NaN
1,01/11/2014,NaN
2,03/11/2014,NaN
3,03/11/2014,NaN
4,03/11/2014,NaN
...,...,...
13309,20/11/2024,NaN
13310,20/11/2024,NaN
13311,20/11/2024,NaN
13312,13/07/2019,21/11/2024


In [57]:
data_groupage['Groupe'].value_counts()

,count
Groupe,
O,6638
A,4295
B,1902
AB,479


In [58]:
data_groupage['determine1'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')

,0
0,31/10/2014
1,01/11/2014
2,03/11/2014
3,03/11/2014
4,03/11/2014
...,...
13309,20/11/2024
13310,20/11/2024
13311,20/11/2024
13312,13/07/2019


In [59]:
wow = '2014 hf \ 20 chhc \ 30'

def extract_date(text) :
  text = str(text).replace(' ' , '')
  text = re.sub('\D' , '' , text)
  match = re.search(r'(\d{2}/\d{1,2}/\d{4})' , text)
  if match:
    return match.group(1)

  else :
    return  text
extract_date(wow)

'20142030'

In [60]:
import re

def search_date(text):
    text = str(text) # Keep original spaces for initial date search
    match = re.search(r'(\d{2}).*(\d{2}).*(\d{4})', text)
    if match:
      return True

    return False


def extract_date(text):
    if search_date(text) :
      match = re.search(r'(\d{2}).*(\d{2}).*(\d{4})', text)
      return match.group(3) +'-' + match.group(2) + '-' + match.group(1)
    return text

extract_date('02DHDHHD/ 05 /GDGDG/HDH2014')

'2014-05-02'

In [61]:
data_groupage['determine1'] = data_groupage['determine1'].apply(extract_date)
data_groupage['determine1']

,determine1
0,2014-10-31
1,2014-11-01
2,2014-11-03
3,2014-11-03
4,2014-11-03
...,...
13309,2024-11-20
13310,2024-11-20
13311,2024-11-20
13312,2019-07-13


In [62]:
data_groupage['determine2'] = data_groupage['determine2'][ data_groupage['determine2'].str.contains(r'.*(\d{1,2}).*(\d{1,2}).*(\d{4})' , na=False) ].apply(extract_date)

<ipython-input-62-3094132754>:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_groupage['determine2'] = data_groupage['determine2'][ data_groupage['determine2'].str.contains(r'.*(\d{1,2}).*(\d{1,2}).*(\d{4})' , na=False) ].apply(extract_date)


In [63]:
data_groupage['determine2'].dropna()

,determine2
391,2015-02-10
864,2015-05-14
1533,2015-09-02
2639,2021-05-19
4717,2021-05-17
...,...
13077,2024-08-05
13099,2024-08-24
13108,2024-08-29
13305,2024-11-19


In [64]:
data_groupage[['determine1' , 'determine2']].dropna()

,determine1,determine2
391,2015-02-10,2015-02-10
864,2015-05-09,2015-05-14
1533,2015-09-02,2015-09-02
2639,2016-02-23,2021-05-19
4717,2017-03-09,2021-05-17
...,...,...
13077,2021-05-19,2024-08-05
13099,2018-01-10,2024-08-24
13108,2018-08-16,2024-08-29
13305,2024-02-25,2024-11-19


In [65]:
data_medecin.columns
#columns_medecin =  ['id', 'nomprenom', 'adresse', 'region', 'specialite', 'telephone1',
#       'telephone2', 'fax', 'gsm']


#data_medecin = data_medecin[columns_medecin]
#data_medecin['code'] = None


data_medecin.columns

Index(['id', 'adresse', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'email', 'fax', 'telephone1', 'nomprenom',
       'Unnamed: 12', 'code', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'region', 'Unnamed: 18', 'commentaire', 'Unnamed: 20', 'Unnamed: 21',
       'telephone2', 'gsm', 'specialite', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'adresse_fromemail',
       'telephone_from_fax', 'telephone_from_telephone1',
       'telephone_from_commentaire', 'telephone_from_telephone2',
       'telephone_from_gsm'],
      dtype='object')

In [71]:
data_medecin[['specialite']].value_counts()

,count
specialite,
CARDIOLOGUE,113
GÉNÉRALISTE,102
GYNÉCOLOGUE,90
MÉDECIN INTERNE,71
UROLOGUE,68
ENDOCRINOLOGUE,66
ONCOLOGUE,66
NEUROLOGUE,49
PÉDIATRE,48


join data

Nom du médecin prescripteur ou informations sur la prescription.

In [67]:
all_data_patient = pd.read_excel('/content/all_data_patient_final_clean.xlsx'  , dtype={'telephone': str})

FileNotFoundError: [Errno 2] No such file or directory: '/content/all_data_patient_final_clean.xlsx'

In [ ]:
all_data_patient.columns

In [ ]:
filter_all_data_patient = all_data_patient[['Chun','nom', 'prenom' , 'adresse', 'region', 'cin', 'dateheure', 'dnaiss',
       'email', 'ba', 'telephone', 'sit', 'ref', 'age' , 'total_comment']]

In [ ]:
data_patient = pd.read_excel('/content/export_patients.xlsx')

In [ ]:
filter_all_data_patient['medecin_nom'] = data_patient['prescri']

In [ ]:
filter_all_data_patient.head(5)

In [ ]:
data_medecin = data_medecin[['id','nomprenom','adresse' ,'region' ,'specialite', 'telephone1'	, 'telephone2'	, 'fax' ,	'gsm']]
data_medecin.head(5)

joining with medecin nom

In [ ]:
merged_data = filter_all_data_patient.merge(data_medecin , how='left' ,left_on='medecin_nom' , right_on='nomprenom' ,suffixes=('', '_doctor'))

In [ ]:
merged_data.head(5)

In [ ]:
columns = filter_all_data_patient.columns.tolist()
columns.extend(['id'])
merged_data = merged_data[columns]

merged_data

In [ ]:
data_groupage = data_groupage[['Groupe' , 'determine1' , 'determine2' ,'id' , 'Rhesus' , 'Vignette' , 'ref' ]]

In [ ]:
#merged_data = filter_all_data_patient.merge(data_medecin , how='left' ,left_on='medecin_nom' , right_on='nomprenom' ,suffixes=('', '_doctor'))
merged_columns= merged_data.columns.tolist()
merged_columns.extend(['Groupe' , 'determine1' , 'determine2' ,'id' , 'Rhesus' , 'Vignette' ])
merged_data =  merged_data.merge(data_groupage , how='left' , left_on='ref' , right_on='ref' , suffixes=('' , '_groupage'))



In [ ]:
data_groupage

In [ ]:
grouped  =  data_groupage.sort_values(by = 'ref').groupby(['ref']).size().reset_index(name='size')
grouped[grouped['size']>1]

In [ ]:
data_groupage[data_groupage['ref']=='14K1008']

In [ ]:
if filter_all_data_patient[filter_all_data_patient['ref']== '14K1008'].shape[0] * data_groupage[  data_groupage['ref'] == '14K1008' ].shape[0] == merged_data[merged_data['ref'] == '14K1008'].shape[0] :
  print("True")

else :
  print("there's a probleme ")

In [ ]:
data_patient.columns

In [ ]:
pd.DataFrame(merged_data.columns , columns = ['column'] )

In [ ]:
isil = '''id
adresse1
adresse2
cin
code_postal
date_creation
date_modification
date_naissance
email
fax
fixe
nom
prenom
sexe
tel
username
ville
champsde_transmission
civilite
commentaire
ipp
mode_transmission
nom_jeune_fille
solde
canal_transmission_id
medecin_id
path_cin
gs
date_determ1
date_determ2
num_determ
ref_determ1
ref_determ2
rhesus
autre_identite
passeport
permis_conduire
titre_sejour
groupe_sanguin
responsable_legal_id'''
mogador = '''-
adresse
region
cin
-
dateheure
-
dnaiss
email
-
-
nomprenom
nomprenom
ba
telephone
-
-
-
sit
observ, comment1, comment2
ref
-
-
-
-
id
-
Groupe
determine1
determine2
id
-
-
Rhesus
-
-
-
-
Vignette
-'''

explication = '''Identifiant unique du patient.
Adresse principale du patient.
Adresse secondaire (si disponible).
Numéro de carte d'identité nationale.
Code postal de la ville du patient.
Date et heure de création de l'enregistrement.
Date et heure de la dernière modification.
Date de naissance du patient.
Adresse e-mail du patient.
Numéro de fax du patient.
Numéro de téléphone fixe.
Nom de famille du patient.
Prénom du patient.
Sexe du patient (M pour masculin, F pour féminin).
Numéro de téléphone mobile.
Nom d'utilisateur associé au patient.
Ville de résidence.
Champs utilisés pour la transmission des données.
Civilité du patient (MR, MME, etc.).
Commentaires supplémentaires.
Identifiant patient permanent.
Mode de transmission des documents médicaux.
Nom de jeune fille (pour les patientes mariées).
Solde du compte du patient (si applicable).
Identifiant du canal de transmission.
Identifiant du médecin référent.
Chemin du fichier contenant la copie de la CIN.
Groupe sanguin du patient.
Date de la première détermination du groupe sanguin.
Date de la deuxième détermination du groupe sanguin.
Numéro de détermination.
Référence de la première détermination.
Référence de la deuxième détermination.
Facteur rhésus du patient (+ ou -).
Autres identités utilisées par le patient.
Numéro de passeport.
Numéro du permis de conduire.
Numéro du titre de séjour (si étranger).
Groupe sanguin du patient (peut être redondant avec gs).
Identifiant du responsable légal (si applicable).'''
explication = explication.split('\n')

mogador = mogador.split('\n')
isil = isil.split('\n')
len(isil) , len(mogador)


dict_mapping = {'isil' : isil , 'mogador' : mogador, 'explication' : explication }

df = pd.DataFrame(dict_mapping)
df

In [ ]:
raw_groupage = pd.read_excel('/content/export_groupages.xlsx')

In [ ]:
pd.DataFrame(raw_groupage.columns , columns = ['columns'])

In [ ]:
merged_data.columns

In [ ]:
merged_data['determine1'] =  pd.to_datetime(merged_data['determine1'] )
merged_data['determine2'] =  pd.to_datetime(merged_data['determine2'] )
merged_data['dateheure'] =  pd.to_datetime(merged_data['dateheure'] )
merged_data['dnaiss'] =  pd.to_datetime(merged_data['dnaiss'] )
merged_data['id'] = merged_data['id'].astype('Int64')



merged_data.dtypes

In [ ]:
merged_data['id']

In [ ]:
unique_data = merged_data.sort_values(by = ['ref' , 'dateheure'] , ascending = [True , False]).drop_duplicates(subset = ['ref'] , keep='first')

In [ ]:
unique_data['id'] = unique_data['id'].apply(lambda x  : np.nan if pd.isna(x) else x )

In [ ]:
unique_data.isna().sum()

In [ ]:
unique_data[unique_data['sit'].isna()][['ref' , 'determine1']]

some of paients there are not in data groupage so they got null in determine1 and the others enregistrements

In [ ]:
data_groupage[['ref' , 'determine1']][data_groupage['ref'] =='15A1596']

In [ ]:
grouped = merged_data['ref'].value_counts().reset_index()
grouped[grouped['count'] == 5 ].sort_values(by=['count'] , ascending =True)

In [ ]:
merged_data[['ref' , 'id']][merged_data['ref'].isin(['24B408' , '15D106'])]

In [ ]:
unique_data.to_csv('merged_data.csv' , index=False)

In [ ]:
data_patient.columns

In [ ]:
data_patient['region'].apply(search_adresse).value_counts()
